<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/main/MVA2024_ex14notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2024 ex14notebookB

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2024

---
## 演習課題: アイスクリーム購入額データの時系列解析
---



In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set_theme()

# statsmodels いろいろ
from statsmodels import tsa
from statsmodels.graphics import tsaplots
from statsmodels.tsa import ar_model
from statsmodels.tsa.arima.model import ARIMA

---
### データの準備


#### データ5: 世帯あたりアイスクリーム・シャーベット購入額



政府統計の総合窓口 e-Stat ( https://www.e-stat.go.jp/ )から入手した，2人以上世帯のアイスクリーム・シャーベットへの支出金額のデータです．2000年1月から2022年10月までの毎月の金額[円]の値から成る時系列です．

このデータは，以下の書籍とウェブページの情報を参考にして作成しました．

- 「時系列解析 ―自己回帰型モデル・状態空間モデル・異常検知―」島田直希，共立出版，2019， https://www.kyoritsu-pub.co.jp/book/b10003204.html
- 「アイスクリームの売れ方」奥村晴彦 https://oku.edu.mie-u.ac.jp/~okumura/stat/160118.html

In [ ]:
# 2000年1月から2022年10月までの毎月のアイスクリーム・シャーベットへの支出金額（世帯あたり）[円]
URL = 'https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/icecream2.csv'
dfIce2 = pd.read_csv(URL, index_col=0)
assert dfIce2.index[0] == '2000-01-01' and dfIce2.index[-1] == '2022-10-01'
dfIce2.index = pd.date_range('2000-01', end='2022-10', freq='MS')
dfIce2

各行に月始の日付が入っていますが，これはデータ作成の都合でそうなっているだけです．各行の値はその月の金額です．

In [ ]:
X_ice = dfIce2['356 アイスクリーム・シャーベット【円】']
fig, ax = plt.subplots(2, 1, figsize=(8, 6))
# 全データをプロット
ax[0].plot(X_ice, '-o', markersize=3)
ax[0].axhline(0, color='gray')
# 3年分だけプロット
ax[1].plot(X_ice[:'2002'], '-o')
ax[1].axhline(0, color='gray')
plt.tight_layout()
plt.show()

---
### 問題1

データ5を見ると，明らかに周期的な変動があります．その周期を調べましょう．

次のセルを実行すると，データ5の時系列の自己相関 $\textrm{Corr}[x_t, x_{t-k}]$ を求めてプロットすることができる．横軸はラグ $k$ を表す．この図を読解して，この時系列の主な季節変動成分の周期は何ヶ月といえるか答えなさい．


In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
tsaplots.plot_acf(X_ice, ax=ax)
ax.set_ylim(-1.1, 1.1)
plt.tight_layout()
plt.show()

---
### 問題2

次の2つのコードセルを実行すると，データ5の時系列にARモデルを当てはめる実験が行える．
ARモデルの次数を $6, 9, 12, 24$ 等と変えて実行し，結果を観察しなさい．
コードセルの下の［補足］も参考にしてね．



In [ ]:
X_train = X_ice[:'2014-12'] # モデルパラメータの推定に使うデータ
X_test  = X_ice['2015-01':] # 予測の検証用に使うデータ

K = 2 # ARモデルの次数

# X_train を使ってモデルパラメータを推定
model = ar_model.AutoReg(X_train, K, old_names=False)
result = model.fit()
print(result.summary())

In [ ]:
# 得られたモデルを使って予測
X_predicted = result.predict(end='2023-12')

# 結果をプロット
fig, ax = plt.subplots(2, 1, figsize=(8, 8))
XX_predicted = X_predicted[:'2014-12']
ax[0].plot(X_train, '-o', label='true', markersize=3)
ax[0].plot(XX_predicted, '-o', color='red', label=f'predicted (K={K})', markersize=3)
ax[0].axhline(0, color='gray')
ax[0].set_ylim(top=2000)
ax[0].legend()
XX_predicted = X_predicted['2015-01':]
resid = X_test - XX_predicted
ax[1].plot(X_test, '-o', label='true', markersize=3)
ax[1].plot(XX_predicted, '-o', color='red', label=f'predicted (K={K})', markersize=3)
ax[1].axhline(0, color='gray')
ax[1].set_ylim(top=2000)
ax[1].legend()
plt.tight_layout()
plt.show()

［補足］

`AutoReg Model Results` の `Log Likelihood` は，モデルの対数尤度の値を示します．この実験では，この対数尤度を最大化するようモデルパラメータを決めているので，この値が大きい方がデータへの当てはまりがよいと言えます．上段のグラフは，モデルパラメータの推定に用いたデータ（ `true` とラベルがついているもの， `X_train`）と，それに対するモデルの予測値（`predicted`，過去 $K$ 時点のデータの値から次の時点の値を予測したもの）を描いたものです．下段のグラフは，2014-12 までの値だけから 2015-01 以降の値を予測したものです．`true` とラベルがついているのがその時期の真の値（`X_test`）です．


ARモデルの次数を $6, 9, 12, 24$ 等と変えて実行し，結果を観察しなさい．また，次のことを考えなさい．

- モデル次数を変えると当てはまりの良さはどう変わるだろうか？
- モデル次数を変えると予測の正しさはどう変わるだろうか？


---
### 問題3

次の2つのコードセルを実行すると，データ5の時系列に SARIMA という時系列モデルを当てはめることができる（解説は後述）．1行目をコメントに従って修正して実行すると，AR(12) モデルに切り替えることもできる．両者の結果を比較しなさい．

In [ ]:
isSARIMA = True  # SARIMA モデルを使う (True) / AR(12) を使う (False) か

X_train = X_ice[:'2014-12']
X_test  = X_ice['2015-01':]

if isSARIMA:
    order = (3, 2, 1)
    sorder = (1, 1, 0, 12)
else:
    order = (12, 0, 0)
    sorder = (0, 0, 0, 0)

# X_train を使ってモデルパラメータを推定
model = ARIMA(X_train, order=order, seasonal_order=sorder)
result = model.fit()
print(result.summary())

In [ ]:
# 得られたモデルを使って予測
X_predicted = result.predict(end='2023-12')

# 結果をプロット
if isSARIMA:
    s = 'predicted (SARIMA)'
else:
    s = 'predicted (AR(12))'
fig, ax = plt.subplots(2, 1, figsize=(8, 8))
XX_predicted = X_predicted[:'2014-12']
ax[0].plot(X_train, '-o', label='true', markersize=3)
ax[0].plot(XX_predicted, '-o', color='red', label=s, markersize=3)
ax[0].axhline(0, color='gray')
ax[0].set_ylim(top=2000)
ax[0].legend()
XX_predicted = X_predicted['2015-01':]
resid = X_test - XX_predicted
ax[1].plot(X_test, '-o', label='true', markersize=3)
ax[1].plot(XX_predicted, '-o', color='red', label=s, markersize=3)
ax[1].axhline(0, color='gray')
ax[1].set_ylim(top=2000)
ax[1].legend()
plt.tight_layout()
plt.show()

SARIMA がどのようなものかの真面目な説明はしませんが，ざっくりいうとこんな感じです．

1. AR と移動平均 (Moving Average, MA) モデルというもうひとつのモデルを組み合わせた，ARMA というモデルがある
1. ARMA を発展させた ARIMA というモデルがある
1. ARIMA に季節変動 (Seasonal Variation) を取り入れたものが SARIMA

AR モデルは定常性を仮定したモデルでしたが，SARIMAは平均の変動や季節変動なども考慮したものとなっています．この実験の条件では，SARIMA の方がデータの当てはまりや予測精度が良い結果が得られます．

